<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import project files files from github 


In [1]:
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

fatal: destination path 'IncrementalLearning' already exists and is not an empty directory.


import libraries


In [2]:
import torch
import os
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
import torch.optim as optim
from torchvision import transforms
import numpy as np

# project classes --------------------------------------------------------------
from dataset.ilcifar100 import ilCIFAR100
from baselines.resnet import resnet32
from info_recorder.info_log import InfoLog
from models_benchmark.benchmark import Benchmark

Initial setup

In [3]:
# Define params

NUM_RUN = 3
SEEDS = [1,2,3]
NUM_EPOCHS = 70
BATCH_SIZE = 64
NUM_CLASS_BATCH = 10
DEVICE = 'cuda'

WEIGHT_DECAY = 1e-5
LR = 2
MOMENTUM = 0.9
MILESTONES = [49, 63]
GAMMA = 0.2

SAVINGS_DIR = "savings"
# Create savings folder
try:
    os.mkdir(SAVINGS_DIR)
except FileExistsError:
    pass

Define transforms

In [4]:
# Define transformations for training
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define transformations for evaluation
test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])

Prepare dataloders for each train, val test subset of the original dataset

In [5]:
# dataloaders dict initialization
dataloaders = {run:{batch:{'train':None, 'val':None, 'test':None} for batch in range(NUM_CLASS_BATCH)} for run in range(NUM_RUN)}

# make dataloaders
for run in range(NUM_RUN):
    
    # download dataset only on the first run
    download = True
    if run != 0:
        download = False
    
    original_training_set = ilCIFAR100(root="cifar", seed=SEEDS[run], transform=train_transform, download=download)
    original_test_set = ilCIFAR100(root="cifar", train=False, seed=SEEDS[run], transform=test_transform, download=download)
    
    for class_batch in range(NUM_CLASS_BATCH):
        
        # get indices
        train_indices = original_training_set.batches[class_batch]['train']
        val_indices = original_training_set.batches[class_batch]['val']
        # test indices are the uninion of test indices of batches seen up to now
        test_indices = []
        for i in range(class_batch+1):
            test_indices += original_test_set.batches[i]

        # make subsets
        train_set = Subset(dataset=original_training_set, indices=train_indices)
        val_set = Subset(dataset=original_training_set, indices=val_indices)
        test_set = Subset(dataset=original_test_set, indices=test_indices)

        # make dataloaders
        train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
        val_dataloader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
        test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

        # add dataloaders to the dataloaders dict
        dataloaders[run][class_batch]['train'] = train_dataloader
        dataloaders[run][class_batch]['val'] = val_dataloader
        dataloaders[run][class_batch]['test'] = test_dataloader

Files already downloaded and verified
Files already downloaded and verified


In [6]:
for run in range(NUM_RUN):
    benchmark = Benchmark(
        num_epochs=NUM_EPOCHS, 
        dataloaders=dataloaders,
        batch_size=BATCH_SIZE,
        saving_folder="savings/fine_tuning")
    
    benchmark.set_criterion(nn.CrossEntropyLoss())
    benchmark.set_infoLog(InfoLog, run)
    benchmark.set_model(resnet32())

    for class_batch in range(NUM_CLASS_BATCH):
        benchmark.log.new_class_batch()
        parameters_to_optimize = benchmark.model.parameters()
        benchmark.set_optimizer(optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY))
        benchmark.set_scheduler(optim.lr_scheduler.MultiStepLR(benchmark.optimizer, milestones=MILESTONES, gamma=GAMMA))

        benchmark.do_class_batch(run, class_batch)

        benchmark.model.add_nodes(10)

RUN 0 ------------------------------------------------------------------------------------

	CLASS BATCH 0

	epoch  0:    train_acc: 0.101  train_loss: 2.937  val_acc: 0.100  val_loss: 2.395
	epoch  1:    train_acc: 0.101  train_loss: 2.374  val_acc: 0.100  val_loss: 2.388
	epoch  2:    train_acc: 0.102  train_loss: 2.372  val_acc: 0.100  val_loss: 2.367
	epoch  3:    train_acc: 0.096  train_loss: 2.376  val_acc: 0.100  val_loss: 2.360
	epoch  4:    train_acc: 0.090  train_loss: 2.378  val_acc: 0.100  val_loss: 2.374

	BEST RESULTS:	val_loss: 2.360,  val_acc: 0.100,  epoch:git push  3
	TEST ACC: 0.100


	CLASS BATCH 1

	epoch  0:    train_acc: 0.106  train_loss: 2.496  val_acc: 0.100  val_loss: 2.383
	epoch  1:    train_acc: 0.103  train_loss: 2.370  val_acc: 0.100  val_loss: 2.406
	epoch  2:    train_acc: 0.104  train_loss: 2.402  val_acc: 0.100  val_loss: 2.408
	epoch  3:    train_acc: 0.092  train_loss: 2.389  val_acc: 0.100  val_loss: 2.406
	epoch  4:    train_acc: 0.100  train_los